In [3]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta

# Carrega variáveis do .env
load_dotenv()

# Lê as variáveis
ESCALLO_HOST = os.getenv("ESCALLO_HOST")
ESCALLO_TOKEN = os.getenv("ESCALLO_TOKEN")

In [ ]:
# ---------------------------- LIGAÇÕES ENTRADA ----------------------------


if not ESCALLO_HOST.startswith("http"):
    ESCALLO_HOST = "http://" + ESCALLO_HOST

# Monta a URL
url = f"{ESCALLO_HOST}/escallo/api/v1/recurso/relatorio/rel001/?dataInicial=2025-05-01&horarioInicial=00:00:01&horarioFinal=23:59:59&filtrarFilhas=0&ultimosDias=30&dataFinal=2025-05-31"

headers = {
    "Authorization": f"Partner {ESCALLO_TOKEN}",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    dados = response.json()

    # Acessa diretamente a lista de registros
    registros = dados.get("data", {}).get("registros", [])

    if not registros:
        print("⚠️ Nenhum registro encontrado na chave 'registros'")
    else:
        df_normalizado = pd.json_normalize(registros)

        # Salvar como JSON
        """ with open("relatorio_entrada.json", "w", encoding="utf-8") as f:
            json.dump(df_normalizado.to_dict(orient="records"), f, ensure_ascii=False, indent=4) """

        # Salvar como Excel
        df_normalizado.to_excel("relatorio_entrada.xlsx", index=False)

        """ print("✅ JSON normalizado salvo como 'relatorio_normalizado.json'") """
        print("✅ Excel salvo como 'relatorio_normalizado.xlsx'")
else:
    print(f"❌ Erro {response.status_code}")
    print(response.text)

# Exibir os dados JSON brutos (opcional)
print(json.dumps(dados, indent=2, ensure_ascii=False))

In [ ]:
# ---------------------------- LIGAÇÕES SAIDA ----------------------------

headers = {
    "Authorization": f"Partner {ESCALLO_TOKEN}",
    "Content-Type": "application/json",
    "Accept": "application/json"
}

def fetch_day_registros(data_str):
    registros_do_dia = []
    pagina = 0
    while True:
        url = f"{ESCALLO_HOST}/escallo/api/v1/recurso/relatorio/rel003/?registros=100&pagina={pagina}"

        payload = {
            "dataInicial": data_str,
            "dataFinal": data_str,
            "horarioInicial": "00:00:01",
            "horarioFinal": "23:59:59",
            "filtrarFilhas": 0
        }

        response = requests.post(url, headers=headers, data=json.dumps(payload))

        if response.status_code != 200:
            print(f"Erro {response.status_code} em {data_str} página {pagina}")
            break

        dados = response.json()
        registros = dados.get("data", {}).get("registros", [])

        if not registros:
            break

        registros_do_dia.extend(registros)
        pagina += 1

    print(f"{len(registros_do_dia)} registros obtidos em {data_str}")
    return registros_do_dia

def daterange(start_date, end_date):
    for n in range((end_date - start_date).days + 1):
        yield (start_date + timedelta(n)).strftime("%Y-%m-%d")

start_date = datetime.strptime("2025-05-01", "%Y-%m-%d")
end_date = datetime.strptime("2025-05-31", "%Y-%m-%d")

todos_registros = []

with ThreadPoolExecutor(max_workers=5) as executor:
    futures = {executor.submit(fetch_day_registros, day): day for day in daterange(start_date, end_date)}

    for future in as_completed(futures):
        registros_dia = future.result()
        todos_registros.extend(registros_dia)

print(f"Total registros coletados: {len(todos_registros)}")

df = pd.json_normalize(todos_registros)
df.to_excel("relatorio_saida_paralelo.xlsx", index=False)
print("✅ Excel salvo com dados paralelos por dia")

In [26]:
def minutos_para_mmss(minutos):
    total_segundos = int(minutos * 60)
    m = total_segundos // 60
    s = total_segundos % 60
    return f"{m}:{s:02d}"

# Lê o Excel
df = pd.read_excel("Relatorio_MaioEN25.xlsx")

# Remove registros com colunas essenciais nulas
df = df.dropna(subset=[
    'filaAtendimentoLigacao.destino',
    'filaAtendimentoLigacao.tempoEspera',
    'filaAtendimentoLigacao.tempoAtendimento',
    'filaAtendimentoLigacao.statusFormatado'
])

# ------------------ Atendidas ------------------

df_atendidas = df[df['filaAtendimentoLigacao.statusFormatado'] == 'Atendida']

# Agrupamento por agente com contagem
agrupado = df_atendidas.groupby('filaAtendimentoLigacao.destino').agg({
    'filaAtendimentoLigacao.tempoEspera': 'mean',
    'filaAtendimentoLigacao.tempoAtendimento': 'mean',
    'filaAtendimentoLigacao.statusFormatado': 'count'
}).reset_index()

agrupado.columns = ['Agente', 'TME (seg)', 'TMA (seg)', 'Total Atendidas']
agrupado['TME (min)'] = agrupado['TME (seg)'] / 60
agrupado['TMA (min)'] = agrupado['TMA (seg)'] / 60
agrupado['TME (mm:ss)'] = agrupado['TME (min)'].apply(minutos_para_mmss)
agrupado['TMA (mm:ss)'] = agrupado['TMA (min)'].apply(minutos_para_mmss)

# Ordena pelo nome
agrupado = agrupado.sort_values('Agente').reset_index(drop=True)

# Média geral
media_geral = pd.DataFrame({
    'Agente': ['MÉDIA GERAL'],
    'TME (seg)': [df_atendidas['filaAtendimentoLigacao.tempoEspera'].mean()],
    'TMA (seg)': [df_atendidas['filaAtendimentoLigacao.tempoAtendimento'].mean()],
    'Total Atendidas': [df_atendidas.shape[0]]
})
media_geral['TME (min)'] = media_geral['TME (seg)'] / 60
media_geral['TMA (min)'] = media_geral['TMA (seg)'] / 60
media_geral['TME (mm:ss)'] = media_geral['TME (min)'].apply(minutos_para_mmss)
media_geral['TMA (mm:ss)'] = media_geral['TMA (min)'].apply(minutos_para_mmss)

# Junta os dados
agrupado = pd.concat([agrupado, media_geral], ignore_index=True)

print("\n✅ Atendidas - TME e TMA por Agente:")
print(agrupado[['Agente', 'Total Atendidas', 'TME (mm:ss)', 'TMA (mm:ss)']])

# ------------------ Abandonadas ------------------

df_abandonadas = df[df['filaAtendimentoLigacao.statusFormatado'].str.contains('Abandonada', na=False)]

agrupado_filas = df_abandonadas.groupby('telefoniaFilaAtendimento.nome').agg({
    'filaAtendimentoLigacao.tempoEspera': 'mean',
    'filaAtendimentoLigacao.statusFormatado': 'count'
}).reset_index()

agrupado_filas.columns = ['Fila', 'TMAbo (seg)', 'Total Abandonadas']
agrupado_filas['TMAbo (min)'] = agrupado_filas['TMAbo (seg)'] / 60
agrupado_filas['TMAbo (mm:ss)'] = agrupado_filas['TMAbo (min)'].apply(minutos_para_mmss)

# Ordena por nome da fila
agrupado_filas = agrupado_filas.sort_values('Fila').reset_index(drop=True)

# Média geral
media_bo = pd.DataFrame({
    'Fila': ['MÉDIA GERAL'],
    'TMAbo (seg)': [df_abandonadas['filaAtendimentoLigacao.tempoEspera'].mean()],
    'Total Abandonadas': [df_abandonadas.shape[0]]
})
media_bo['TMAbo (min)'] = media_bo['TMAbo (seg)'] / 60
media_bo['TMAbo (mm:ss)'] = media_bo['TMAbo (min)'].apply(minutos_para_mmss)

# Junta os dados
agrupado_filas = pd.concat([agrupado_filas, media_bo], ignore_index=True)

print("\n🚫 Abandonadas - TMAbo por Fila:")
print(agrupado_filas[['Fila', 'Total Abandonadas', 'TMAbo (mm:ss)']])


✅ Atendidas - TME e TMA por Agente:
                                          Agente  Total Atendidas TME (mm:ss)  \
0                                           9910             2056        0:02   
1               Suporte - Alison da Silva (4025)              238        0:18   
2    Suporte - Eduardo Tomaz (Estagiário) (4008)              239        0:24   
3               Suporte - Gabriel Marques (4005)              244        0:19   
4                  Suporte - Gabriel Rosa (4006)              302        0:14   
5                    Suporte - João Gomes (4020)              249        0:16   
6                   Suporte - João Miyake (4004)              181        0:19   
7                 Suporte - Kaiky Leandro (4001)               96        0:15   
8   Suporte - Pedro Henrique (Estagiário) (4002)              223        0:16   
9   Suporte - Rubens Ribeiro (Estagiário) (4009)              202        0:14   
10               Técnico - Felipe Gabriel (5001)                6       

In [27]:
agrupado

,Agente,TME (seg),TMA (seg),Total Atendidas,TME (min),TMA (min),TME (mm:ss),TMA (mm:ss)
0,9910,2.188716,547.116732,2056,0.036479,9.118612,0:02,9:07
1,Suporte - Alison da Silva (4025),18.928571,272.180672,238,0.315476,4.536345,0:18,4:32
2,Suporte - Eduardo Tomaz (Estagiário) (4008),24.782427,269.405858,239,0.413040,4.490098,0:24,4:29
3,Suporte - Gabriel Marques (4005),19.409836,236.467213,244,0.323497,3.941120,0:19,3:56
4,Suporte - Gabriel Rosa (4006),14.953642,293.466887,302,0.249227,4.891115,0:14,4:53
5,Suporte - João Gomes (4020),16.799197,272.096386,249,0.279987,4.534940,0:16,4:32
6,Suporte - João Miyake (4004),19.613260,383.497238,181,0.326888,6.391621,0:19,6:23
7,Suporte - Kaiky Leandro (4001),15.697917,604.156250,96,0.261632,10.069271,0:15,10:04
8,Suporte - Pedro Henrique (Estagiário) (4002),16.672646,150.502242,223,0.277877,2.508371,0:16,2:30
9,Suporte - Rubens Ribeiro (Estagiário) (4009),14.168317,372.163366,202,0.236139,6.202723,0:14,6:12


In [28]:
agrupado_filas

,Fila,TMAbo (seg),Total Abandonadas,TMAbo (min),TMAbo (mm:ss)
0,Aprimorar,83.000000,3,1.383333,1:23
1,Aprimorar - 100%,0.000000,1,0.000000,0:00
2,Geral,49.840909,88,0.830682,0:49
3,Rompimentos,214.666667,3,3.577778,3:34
4,Suporte,167.909091,44,2.798485,2:47
5,Suporte - Técnicos,169.969697,66,2.832828,2:49
6,Suporte PJ,172.550000,20,2.875833,2:52
7,MÉDIA GERAL,121.493333,225,2.024889,2:01


In [29]:
# Função para converter minutos decimais em mm:ss
def minutos_para_mmss(minutos):
    total_segundos = int(minutos * 60)
    m = total_segundos // 60
    s = total_segundos % 60
    return f"{m}:{s:02d}"

# Lê o Excel com os dados de saída
df_saida = pd.read_excel("Relatorio_MaioSA25.xlsx")

# Remove registros nulos
df_saida = df_saida.dropna(subset=['ligacao.nomeAgenteOrigem', 'ligacao.tempoTotal'])

# Agrupa por agente: conta total de ligações e média do tempo de conversa
agrupado_saida = df_saida.groupby('ligacao.nomeAgenteOrigem').agg({
    'ligacao.tempoTotal': ['count', 'mean']
}).reset_index()

# Renomeia as colunas
agrupado_saida.columns = ['Agente', 'Total Ligações', 'TMC (min)']

# Converte tempo médio para minutos e mm:ss
agrupado_saida['TMC (min)'] = agrupado_saida['TMC (min)'] / 60
agrupado_saida['TMC (mm:ss)'] = agrupado_saida['TMC (min)'].apply(minutos_para_mmss)

# Ordena por TMC decrescente
agrupado_saida = agrupado_saida.sort_values(by='TMC (min)', ascending=False).reset_index(drop=True)

# Mostra resultado por agente
print("\n📞 Ligações de Saída por Agente:")
print(agrupado_saida[['Agente', 'Total Ligações', 'TMC (mm:ss)']])

# Média geral do tempo de conversa
tmc_geral = df_saida['ligacao.tempoTotal'].mean() / 60
print(f"\n📊 TMC Geral (Tempo Médio de Conversa): {minutos_para_mmss(tmc_geral)} (mm:ss)")


📞 Ligações de Saída por Agente:
                                   Agente  Total Ligações TMC (mm:ss)
0                         Técnico - Ronan               2        6:26
1                 Técnico - Thales Mikael              19        3:28
2               Suporte - Alison da Silva             211        3:23
3      Suporte - João Miyake (Estagiário)             180        2:54
4    Suporte - Eduardo Tomaz (Estagiário)             300        2:52
5                   Suporte - João Miyake              77        2:49
6    Suporte - Kaiky Leandro (Estagiário)             382        2:43
7                    Técnico - Lucas Maia              20        2:37
8                  Suporte - Gabriel Rosa             351        2:32
9                    Suporte - João Gomes             409        2:25
10  Suporte - Pedro Henrique (Estagiário)             324        2:18
11              Suporte - Gabriel Marques             390        2:14
12  Suporte - Rubens Ribeiro (Estagiário)             515

In [30]:
agrupado_saida

,Agente,Total Ligações,TMC (min),TMC (mm:ss)
0,Técnico - Ronan,2,6.441667,6:26
1,Técnico - Thales Mikael,19,3.478947,3:28
2,Suporte - Alison da Silva,211,3.399763,3:23
3,Suporte - João Miyake (Estagiário),180,2.913519,2:54
4,Suporte - Eduardo Tomaz (Estagiário),300,2.867556,2:52
5,Suporte - João Miyake,77,2.819264,2:49
6,Suporte - Kaiky Leandro (Estagiário),382,2.731588,2:43
7,Técnico - Lucas Maia,20,2.630000,2:37
8,Suporte - Gabriel Rosa,351,2.535090,2:32
9,Suporte - João Gomes,409,2.417889,2:25
